In [39]:
import pandas as pd
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from scripts import utils

In [40]:
courses_cg = [202, 250]
courses_ee = [251, 424, 551]
courses = courses_cg + courses_ee
semester = "F2025"

Open all relevant files

In [41]:
data_dir = os.path.join(project_root, 'data')
course_data_dir = os.path.join(data_dir, 'student_emails_and_ids')

ee_file = "F2025_ee_clean.csv"
cg_file = "F2025_cg_clean.csv"
courses_files = {}
for course in courses:
    courses_files[course] = f"{semester}_COMP{course}.csv"

ee_data_path = os.path.join(data_dir, 'clean', ee_file)
cg_data_path = os.path.join(data_dir, 'clean', cg_file)
courses_data_path = {}
for course in courses:
    courses_data_path[course] = os.path.join(course_data_dir, courses_files[course])

Open the cleaned data files

In [42]:
ee_data = pd.read_csv(ee_data_path, header=[0,1], index_col=0)
cg_data = pd.read_csv(cg_data_path, header=[0,1], index_col=0)
data_all = [ee_data, cg_data]

for d in data_all:
    utils.rebuild_multiindex(d)

Open the course data file and extract IDs

In [43]:
student_ids = {}
for course in courses:
    course_data = pd.read_csv(courses_data_path[course])
    ids = course_data.OrgDefinedId.str.strip("#")
    student_ids[course] = list(pd.to_numeric(ids))

### Duplicates
Check if any student answered the survey twice

In [44]:
# Check duplicate students within EE or CG
no_duplicates = True
for d in data_all:
    students = set(d["StudentID"])
    if len(students) != len(d):
        print("One or more students responded in the survey multiple times within the same group.")
        
        # TODO: code for handling duplicate 
        no_duplicates = False

if no_duplicates:
    print("No duplicate students found within the EE or CG groups.")

No duplicate students found within the EE or CG groups.


In [45]:
# Check duplicate students across EE and CG
no_duplicates = True
all_students = set(pd.concat([ee_data["StudentID"], cg_data["StudentID"]]))
if len(all_students) != len(ee_data) + len(cg_data):
    print("One or more students responded in both EE and CG surveys")
    
    # TODO: code for handling duplicate students
    no_duplicates = False

if no_duplicates:
    print("No duplicate students found across the EE and CG groups.")

No duplicate students found across the EE and CG groups.


### Student validation
Verify that students who answered survey are enrolled in the courses they selected in the survey.

1. EE data

In [46]:
for i, row in ee_data.iterrows():
    id = row["StudentID"][""]
    for course in courses_ee:
        course_name = "COMP" + str(course)
        if row["EE course"][course_name] == 1:
            if id not in student_ids[course]:
                print(f"Student doesn't exist") # modify to add student ID

Student doesn't exist


2. CG data

In [47]:
for i, row in cg_data.iterrows():
    id = row["StudentID"][""]
    for course in courses_cg:
        course_name = "COMP" + str(course)
        if row["CG course"][course_name] == 1:
            if id not in student_ids[course]:
                print(f"Student doesn't exist") # modify to add student ID

Student doesn't exist
